In [ ]:
'configs/'base.yml'

In [6]:
n_steps

7

In [14]:
timestep_start = 0
timestep_end   = 30
n_chunks = 4
if abs(timestep_start-timestep_end)%n_chunks==0:
    n_steps = int(abs(timestep_start-timestep_end)/n_chunks)
else:
    n_steps = int(abs(timestep_start-timestep_end)/n_chunks)+1
    
[f'{i*n_steps}_{(i+1)*n_steps}' if (i+1)*n_steps<timestep_end else f'{i*n_steps}_{timestep_end}' for i in range(n_chunks)]

['0_8', '8_16', '16_24', '24_30']

In [45]:
from pathlib import Path, PosixPath 
DATA_PROCESSED = Path('data/processed')

path_proc='%s/%s/'%(DATA_PROCESSED,'EXP1')
path_proc1= DATA_PROCESSED / 'EXP00_test' #/ 'properties/properties'
#PosixPath(path_proc1.parent.resolve()).exists() #.is_dir()#mkdir()

[i.name for i in list(path_proc1.resolve().glob('E*'))]

[]

In [46]:
path_proc1.resolve()

PosixPath('/proj/marine/users/x_bensc/Analysis/ECOdiagnostics/ECOprocessing/data/processed/EXP00_test')

In [2]:
import xarray as xr
import sys, os
sys.path.insert(1, os.path.abspath('..') )
ds1 = xr.open_dataset('../ECOanalysis/data/EXP00_test/EXP00_test_proc_0_8.nc')
ds2 = xr.open_dataset('../ECOanalysis/data/EXP00_test/EXP00_test_pre_8_16.nc')

In [2]:
import sys, os
sys.path.insert(1, os.path.abspath('..') )
from ECOdiagnostics import utils

             

kwargs_proc, kwargs_pre, kwargs_sim = utils.config_parser(config_path='Configs/EXP00_5d50', sub_config='sub_5d50')
kwargs_proc['files_pre']='%s/%s/%s_pre'%(kwargs_proc['output_path'],kwargs_proc['exp_out'],kwargs_proc['exp_out'])

Configs/EXP00_5d50/base.yml
Update processing configuration with sub_5d50


"datapaths_og = datapaths\nfor p in range(len(datapaths)):\n    datapaths[p] = datapaths[p][:-3]+'_temp.nc'\n    print('Temporary save to %s'%(datapaths[p]))\nsave()\n\nfor f in files_to_remove:\n    print('Remove ', f)\n    os.remove(f)\nfor p in range(len(datapaths)):\n    print('Rename to %s'%(datapaths_og[p]))\n    os.rename(datapaths[p],datapaths_og[p])"

In [19]:
test={}
def get_namelist(path = None):
    import f90nml
    nml = f90nml.read(path+'/namelist_ref')
    nml_cfg = f90nml.read(path+'/namelist_cfg_run')
    return {**nml, **nml_cfg}
test = {**test,**get_namelist(path="/proj/marine/users/x_bensc/NEMOGCM/tests/BASIN_compiled_v0.1.2/EXP00_d/")}

In [27]:
import glob
filespath_prefix = '../ECOanalysis/data/EXP00_test/EXP00_test_pre'

if not glob.glob(filespath_prefix+'*'):
    print('skip')
else:
    time_start = []
    time_stop  = []
    for f in glob.glob(filespath_prefix+'*'): 
       file_name_parts = os.path.split(f[:-3])[1].split('_')
       time_start.append(int(file_name_parts[-2]))
       time_stop.append(int(file_name_parts[-1]))

In [30]:
input_text = 'Action required!\n\
                There is existing data of the selected simulation and timesteps,\n\
                but override is disabled.\n\
                Remove everything with timestamp later time_start? [y/n]'
choice = ''
while choice not in ['y','yes','n','no']:
    choice = input(input_text).lower()


In [8]:
# Importing the library
import psutil
import os
# Getting % usage of virtual_memory ( 3rd field)
#print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
#print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

process = psutil.Process(os.getpid())
#   mem = process.get_memory_info()[0] / float(2 ** 20)
mem = process.memory_full_info()
print('RAM', mem)
#print('RAM', mem[1]/1000000000)

RAM 0.058753024
RAM 1.057411072


In [13]:
import yaml
from utils import drop_from_dict, split_by_chunks, str_or_none, get_number_of_chunks

with open("Configs/EXP00_5d50/base.yml", "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    
kwargs_proc = cfg['processing']
kwargs_pre = cfg['preprocessing']
kwargs_sim = cfg['simulation']


In [1]:
args={'c':'sub_5d50'}
sub_conf = args['c'] if args['c'][-4:] == '.yml' else args['c']+'.yml'
if os.path.exists('Configs/EXP00_5d50/%s'%(sub_conf)):
    print('exists')
with open('Configs/EXP00_5d50/%s'%(sub_conf), 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
if 'processing' in cfg: kwargs_proc = {**kwargs_proc,**cfg['processing']}
if 'preprocessing' in cfg: kwargs_pre = {**kwargs_pre,**cfg['preprocessing']}
if 'simulation' in cfg: kwargs_sim = {**kwargs_sim,**cfg['simulation']}

exists


NameError: name 'yaml' is not defined

In [24]:
int(round(ds.t.size/kwargs_proc['n_chunks'],0))

2

In [25]:
import dask.array as da
import xarray as xr

ds=xr.open_dataset('../ECOanalysis/data/EXP_test_n0.nc',decode_times=False)#,chunks={'t':10})

n_chunks=3
#ds = ds.chunk({'t':ds.t.size/n_chunks})
ds = ds.chunk({'t':int(round(ds.t.size/kwargs_proc['n_chunks'],0))})

In [18]:
import glob
#os.path.exists("../ECOanalysis/data//EXP00_d*")
for f in glob.glob("../ECOanalysis/data//EXP00_d*"):
    print(f)

../ECOanalysis/data/EXP00_d_n0.nc
../ECOanalysis/data/EXP00_d_n1.nc
../ECOanalysis/data/EXP00_d.nc
../ECOanalysis/data/EXP00_d_n8.nc


In [7]:
from utils import create_chunks_path, split_by_chunks
kwargs_pre['export_to'] = kwargs_proc['output_path']

#ds_chunks = split_by_chunks(ds)
ds_chunks, chunk_slices, chunk_paths = split_by_chunks(ds, path_prefix="%s/%s"%(kwargs_pre['export_to'],kwargs_proc['exp_out']))
#chunk_paths_out = create_chunks_path(ds, kwargs_proc['n_chunks'])
chunk_paths

['../ECOanalysis/data//EXP00_test_raw_t0_10.nc',
 '../ECOanalysis/data//EXP00_test_raw_t10_20.nc',
 '../ECOanalysis/data//EXP00_test_raw_t20_30.nc']

In [3]:
from xnemogcm import open_nemo_and_domain_cfg, get_metrics
from datetime import datetime
startTime = datetime.now()
ds_raw=None
if ds_raw is None:
    pass
else:
    del ds_raw
ds_raw = open_nemo_and_domain_cfg(
        nemo_files=kwargs_proc['nemo_path']+kwargs_proc['exp_in'],
        domcfg_files=kwargs_proc['domain_path']+kwargs_proc['exp_in'],
        nemo_kwargs={'decode_times':False}#,'parallel':True,'chunks':{'x_c':10,'x_f':10}}#,'y_c':20,'y_f':20,'z_c':18,'z_f':18,}}
    )

print(datetime.now() - startTime)

0:02:10.799295


In [4]:
ds_raw

<xarray.Dataset>
Dimensions:               (z_f: 36, axis_nbounds: 2, t: 3600, y_c: 40, x_c: 20,
                           z_c: 36, x_f: 20, y_f: 40)
Coordinates: (12/19)
  * z_f                   (z_f) float64 -0.5 0.5 1.5 2.5 ... 31.5 32.5 33.5 34.5
    time_centered         (t) float64 dask.array<chunksize=(3600,), meta=np.ndarray>
  * t                     (t) float64 -5.907e+10 -5.907e+10 ... -5.751e+10
  * x_c                   (x_c) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
  * y_c                   (y_c) int64 0 1 2 3 4 5 6 7 ... 33 34 35 36 37 38 39
  * z_c                   (z_c) int64 0 1 2 3 4 5 6 7 ... 29 30 31 32 33 34 35
    ...                    ...
    gphit                 (y_c, x_c) float64 dask.array<chunksize=(11, 10), meta=np.ndarray>
    gphiu                 (y_c, x_f) float64 dask.array<chunksize=(11, 10), meta=np.ndarray>
    gphiv                 (y_f, x_c) float64 dask.array<chunksize=(11, 10), meta=np.ndarray>
    gphif                 (y_f, x_f) float64 dask.array<chunksize=(11, 10), meta=np.ndarray>
    gdept_0               (z_c, y_c, x_c) float64 dask.array<chunksize=(36, 11, 10), meta=np.ndarray>
    gdepw_0               (z_f, y_c, x_c) float64 dask.array<chunksize=(36, 11, 10), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds
Data variables: (12/79)
    depthw_bounds         (z_f, axis_nbounds) float32 dask.array<chunksize=(36, 2), meta=np.ndarray>
    time_centered_bounds  (t, axis_nbounds) float64 dask.array<chunksize=(3600, 2), meta=np.ndarray>
    t_bounds              (t, axis_nbounds) float64 dask.array<chunksize=(3600, 2), meta=np.ndarray>
    e3w                   (t, z_f, y_c, x_c) float32 dask.array<chunksize=(3600, 36, 40, 20), meta=np.ndarray>
    woce                  (t, z_f, y_c, x_c) float32 dask.array<chunksize=(3600, 36, 40, 20), meta=np.ndarray>
    woce_eiv              (t, z_f, y_c, x_c) float32 dask.array<chunksize=(3600, 36, 40, 20), meta=np.ndarray>
    ...                    ...
    top_level             (y_c, x_c) int32 dask.array<chunksize=(11, 10), meta=np.ndarray>
    bottom_level          (y_c, x_c) int32 dask.array<chunksize=(11, 10), meta=np.ndarray>
    stiffness             (y_c, x_c) float64 dask.array<chunksize=(11, 10), meta=np.ndarray>
    ht_0                  (y_c, x_c) float64 dask.array<chunksize=(11, 10), meta=np.ndarray>
    hu_0                  (y_c, x_f) float64 dask.array<chunksize=(11, 10), meta=np.ndarray>
    hv_0                  (y_f, x_c) float64 dask.array<chunksize=(11, 10), meta=np.ndarray>
Attributes: (12/13)
    name:                    NEMO dataset
    description:             Ocean grid variables, set on the proper positions
    title:                   Ocean grid variables
    Conventions:             CF-1.6
    timeStamp:               2022-Dec-05 12:57:50 GMT
    uuid:                    47f91617-7a1c-4188-beb5-55c652f428ca
    ...                      ...
    DOMAIN_size_global:      [20 40]
    DOMAIN_halo_size_start:  [0 0]
    DOMAIN_halo_size_end:    [0 0]
    DOMAIN_type:             BOX
    nn_cfg:                  2
    cn_cfg:                  BASIN

In [6]:
ds_raw = ds_raw.chunk(ds_raw.dims).isel(t=slice(0,360))

kwargs_proc['n_chunks'] = get_number_of_chunks(ds_raw, **kwargs_proc)
ds_raw = ds_raw.chunk({'t':int(round(ds_raw.t.size/kwargs_proc['n_chunks'],0))})

check_exists='%s/%s/%s_pre_t'%(kwargs_proc['output_path'],kwargs_proc['exp_out'],kwargs_proc['exp_out'])
ds_chunks, kwargs_proc['chunk_slices'], kwargs_proc['chunk_paths'] = split_by_chunks(ds_raw, path_prefix=check_exists)

In [13]:
ds_raw.nbytes

1088592416

In [31]:
import xarray as xr
kwargs_proc['files_pre']='%s/%s/%s_pre_t'%(kwargs_proc['output_path'],kwargs_proc['exp_out'],kwargs_proc['exp_out'])

ds = xr.open_mfdataset(kwargs_proc['files_pre']+'*', decode_times=False, parallel = True)#, chunks={'t':})  



In [39]:
import sys
sys.path.insert(1, os.path.abspath('..') )
import ECOdiagnostics as eco
import xgcm
from xbasin.stream_functions import compute_moc

_metrics = {('X',): ['e1t', 'e1u', 'e1v', 'e1f'],
     ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'],
     ('Z',): ['e3t', 'e3u', 'e3v', 'e3w']}
    
bd = {'boundary': 'fill', 'fill_value': 0}
grid = xgcm.Grid(ds, metrics=_metrics, periodic=False)
grid_ops = eco.Grid_ops(grid, boundary=bd, maskargs={'mask':'nan'})

ds['e3tm_1d']=grid_ops.average(ds.e3t,['Y','X'])
grid = grid_ops._update({'Z':ds['e3tm_1d']})
    
psi = compute_moc(ds.vo,grid)
psi = psi.chunk({d: ds.chunks[d] for d in psi.dims})#.chunk(ds.chunks)
ds['psi'] = psi
ds['psi_maxz'] = psi.max('z_f')
ds['psi_dmoc'] = ds['psi_maxz'].isel(y_f=slice(9,None)).max('y_f')
ds['psi_tmoc'] = ds['psi_maxz'].isel(y_f=slice(None,9)).max('y_f')

In [40]:
ds.chunks

Frozen({'t': (112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 8), 'z_f': (36,), 'axis_nbounds': (2,), 'y_c': (40,), 'x_c': (20,), 'z_c': (36,), 'x_f': (20,), 'y_f': (40,)})